# Imports

In [1]:
import sys
sys.path.append('/home/jovyan/work/sem-covid/')
sys.path = list(set(sys.path))

import os
os.getcwd()
os.chdir('/home/jovyan/work/sem-covid/')

import spacy
nlp = spacy.load('en_core_web_sm')

import mlflow
import numpy as np
import gensim
import pandas as pd

from sem_covid.services.store_registry import store_registry
from sem_covid.entrypoints.notebooks.topic_modeling.topic_modeling_wrangling.token_management import select_pos
from gensim.models import LdaModel

2021-09-18 07:34:56.395486: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-18 07:34:56.395507: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Global variables

In [2]:
EXPERIMENT_ID = '101'
BUCKET_NAME = 'mlflow'

# Fetch the data

In [3]:
es_index_store = store_registry.es_index_store()
docs_and_embeddings = es_index_store.get_dataframe(index_name='fs_doc_embeddings_for_topic_modeling')

100% (4126 of 4126) |####################| Elapsed Time: 0:00:06 Time:  0:00:06


In [4]:
docs_and_embeddings.head()

,Date,Document_source,txt,embedding
_id,,,,
027fba8f0b404718bda1c347e4a0cb750d9bb92c63afc8eb30e0091c01ef2688,2021-02-24,eu_cellar,opinion european economic social committee pro...,"[-0.0300699547, -0.0067481417, 0.5634582043, -..."
0114f84fb2837020f8e54710b07de6f05039de056ed49244e1c905b8d1ca6ebd,2021-06-17,eu_cellar,proposal council implement decision approval a...,"[-0.1624983698, 0.0182262585, 0.4614146054, -0..."
02bd88b95042ca0d3bb24689f9f51d5751091a3a90cde36becb9b73d33892a68,2020-03-04,eu_cellar,proposal decision european parliament council ...,"[-0.2034286261, 0.0313053504, 0.5064448118, -0..."
030340e1dac5382a9379488c63e7c6c0cb2ec305f62bf18a38a76a0d23028b42,2020-09-18,eu_cellar,commission staff work document equip europe wo...,"[-0.3065283895, 0.0841537565, 0.6065080166, 0...."
02f0d561c45748182441499f33b5cdda02b1306c8cd06d91ac02ea54f3a917c4,2020-11-23,eu_cellar,council decision november position take behalf...,"[-0.4295326769, 0.0872802585, 0.1653565913, -0..."


# Main

BERTopic

In [5]:
best_run = mlflow.search_runs(
    experiment_ids=EXPERIMENT_ID,
    order_by=['parameters.freq_topic_minus_1 ASC']
).iloc[0]

In [6]:
bert_model = store_registry.minio_feature_store(BUCKET_NAME).get_features(
    features_name=f'{EXPERIMENT_ID}/{best_run.run_id}/artifacts/model/model.pkl')

In [7]:
bert_probabilities = bert_model.transform(documents=docs_and_embeddings['txt'], embeddings=np.array(list(docs_and_embeddings['embedding'])))[1]

In [8]:
bert_probabilities.shape

(4126, 54)

In [9]:
bert_probabilities[0]

array([6.88117014e-16, 1.00000000e+00, 2.64811840e-16, 1.57644733e-16,
       9.45653202e-17, 3.62201398e-16, 3.18444451e-16, 2.88095524e-16,
       1.21102044e-16, 2.20375551e-16, 5.06123343e-17, 1.26680193e-16,
       5.77862578e-17, 6.24136946e-17, 6.80956418e-17, 3.29367734e-17,
       4.42825428e-17, 9.40256076e-17, 4.51283431e-17, 6.81653346e-17,
       2.23025167e-17, 5.41571084e-17, 5.52405482e-17, 1.34936889e-16,
       2.41943926e-17, 6.40040117e-17, 5.25930206e-17, 6.97536422e-17,
       5.29826074e-17, 9.99841600e-17, 7.06594699e-17, 4.56421168e-17,
       1.52796202e-16, 3.57143171e-17, 3.04016971e-17, 6.96292382e-17,
       2.25722060e-17, 7.56292444e-17, 2.20911669e-17, 4.38422700e-17,
       3.16286494e-17, 2.48335540e-17, 6.27197459e-17, 7.66224082e-17,
       3.63242021e-17, 3.33663471e-17, 4.35735520e-17, 5.16354311e-17,
       6.01793459e-17, 4.96143055e-17, 3.59519856e-17, 1.41341245e-16,
       7.61866897e-17, 6.20920239e-17])

In [10]:
es_index_store.put_dataframe(index_name='fs_bert_topic_based_doc_embeddings',
                             content=pd.DataFrame({'date': docs_and_embeddings['Date'], 
                                                   'document_source': docs_and_embeddings['Document_source'], 
                                                   'txt': docs_and_embeddings['txt'],
                                                   'topic_based_embedding': bert_probabilities.tolist()}))

 62% (2560 of 4126) |############        | Elapsed Time: 0:00:00 ETA:   0:00:00

4126

LDA

In [11]:
spacy_corpus = docs_and_embeddings['txt'].apply(nlp)

noun_corpus = spacy_corpus.apply(select_pos, pos='NOUN')
noun_corpus = noun_corpus.apply(lambda x: list(map(lambda docs: docs.lemma_,x)))

noun_dictionary = gensim.corpora.Dictionary(noun_corpus)
noun_gensim_corpus = [noun_dictionary.doc2bow(docs) for docs in noun_corpus]

In [12]:
noun_lda_model = LdaModel(corpus=noun_gensim_corpus, num_topics=5, id2word=noun_dictionary, chunksize=30, random_state=40)

In [13]:
lda_probabilities = []

for doc_topics_distrib in noun_lda_model.get_document_topics(noun_gensim_corpus):
    probs = [item[1] for item in doc_topics_distrib]
    lda_probabilities.append(probs)

In [14]:
len(lda_probabilities)

4126

In [15]:
lda_probabilities[0]

[0.36552876, 0.10348599, 0.02175473, 0.023701508, 0.48552904]

In [16]:
es_index_store.put_dataframe(index_name='fs_lda_topic_based_doc_embeddings',
                             content=pd.DataFrame({'date': docs_and_embeddings['Date'], 
                                                   'document_source': docs_and_embeddings['Document_source'], 
                                                   'txt': docs_and_embeddings['txt'], 
                                                   'topic_based_embedding': lda_probabilities}))

 83% (3448 of 4126) |################    | Elapsed Time: 0:00:00 ETA:   0:00:00

4126